In [ ]:
from library_dicom.rtss_processor.model.MaskBuilder_RTSS import MaskBuilder_RTSS

import matplotlib.pyplot as plt
import numpy as np

## RTSTRUCT file -> Mask 

In [ ]:
rtss_path = 'abs//rtss//path'
#RTSS in CT MODALITY 
ct_path = 'abs//ct//path'
pet_path = 'abs//pet//path'

### CT RTSS to PET array

In [ ]:
mask_builder_rtss = MaskBuilder_RTSS(rtss_path, pet_path)

In [ ]:
from time import process_time
t1_start = process_time()
mask_4D = mask_builder_rtss.rtss_to_4D_mask()
t1_stop = process_time() 
print("diff début fin :", t1_stop-t1_start)

In [ ]:
print("Number of ROI in RTSS : ", mask_builder_rtss.number_of_roi)
print("Size of array in PET Modality : ", mask_4D.shape)

In [ ]:
print('Slice from 4D array ')
slice = 
plt.imshow(mask_4D[:,:,slice,0]) #0 pour ROI 1
plt.show()

### CT RTSS to CT array 

In [ ]:
mask_builder_rtss = MaskBuilder_RTSS(rtss_path, ct_path)
from time import process_time
t1_start = process_time()
mask_4D = mask_builder_rtss.rtss_to_4D_mask()
t1_stop = process_time() 
print("diff début fin :", t1_stop-t1_start)

In [ ]:
print("Number of ROI in RTSS : ", mask_builder_rtss.number_of_roi)
print("Size of array in CT Modality : ", mask_4D.shape)

In [ ]:
print('Slice from 4D array ')
slice = 
plt.imshow(mask_4D[:,:,slice,0]) #0 pour ROI 1
plt.show()

## Mask -> RTSTRUCT file 


In [ ]:
from library_dicom.rtss_processor.model.RTSS_Writer import RTSS_Writer
import pydicom

In [ ]:
dataset = {}
for i in range(1,11):
    data = {}
    data['ROIName'] = str(i)
    data['ROIGenerationAlgorithm'] = 'SEMIAUTOMATIC'
    dataset[i] = data
print(dataset)    

In [ ]:
new_rtss = RTSS_Writer(mask_4D, ct_path, dataset)

In [ ]:
filename = ''
directory_path = ''
new_rtss.save_file(filename, directory_path)